![INSA](https://www.insa-lyon.fr/sites/all/themes/insa/logo.png)

# GI-5-DSC - Data Science: Regression
***

The objective of this part of the tutorial is to continue the analysis of velo'v data and to experiment with artificial intelligence methods for data prediction using regression methods
We will add a layer of weather data and use regression methods to predict velo'v availability.


## 1. Importation des libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import folium
import plotly
import plotly.express as px
import geopandas

import seaborn as sn

import sklearn.cluster

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## 2. Getting the data

First, the dataset must be loaded. 
On the one hand the data set containing the locations of the stations and on the other hand the usage history. 
The second one has been modified during the previous session. In order not to have to redo all the processing you can retrieve the `data-bikes-2.zip` archive directly.


All the data used in this tutorial is available on the [git repository](https://github.com/ludovicmoncla/insa-5gi-dsc-tutorials/tree/main/data) and on [Moodle](https://moodle.insa-lyon.fr/course/view.php?id=4628). 


* Download the datasets
1. data-stations.zip
2. data-bikes-3.zip



### 2.1. Google Colab users : download datasets

In [ ]:
! wget https://perso.liris.cnrs.fr/lmoncla/GI-5-DSC/data-bikes-3.zip
! wget https://perso.liris.cnrs.fr/lmoncla/GI-5-DSC/data-stations.zip

path = ''

### 2.2. Loading the data

As last time, to load the data you just have to use the method [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) from the `Pandas` library. 
It takes as a parameter the path of the file you want to load. This file can be of 2 formats, either directly a CSV file, or a ZIP file containing a CSV. In our case it is therefore unnecessary to unzip the previously downloaded archives.

In [ ]:
## We load the data from the stations into a dataframe
df_stations = pd.read_csv('data-stations.zip')

## We now load the dataframe with the history data
df_bikes = pd.read_csv('data-bikes-3.zip')

In [ ]:
## Display the first rows
df_stations.head()

In [ ]:
## Display the first rows
df_bikes.head()

In [ ]:
# Reduce the size in memory
df_bikes['time'] = pd.to_datetime(df_bikes['time']) 
df_bikes[['year', 'daily_departure', 'daily_arrival']] = df_bikes[['year', 'daily_departure', 'daily_arrival']].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')


## 3. Adding an additional data layer: the weather


### 3.1 Loading the data

The `data-weather-lyon.csv` file contains the weather data by hour for the year 2021 for Lyon.
You can download this file from [Moodle](https://moodle.insa-lyon.fr/mod/folder/view.php?id=180291) or from the [git repository](https://github.com/ludovicmoncla/insa-5gi-dsc-tutorials/blob/main/data/data-weather-lyon.csv)


The weather dataset corresponds to the `NASA/POWER CERES/MERRA2 Native Resolution Hourly Data` dataset retrieved through the NASA POWER Project API : https://power.larc.nasa.gov/.


In [ ]:
## You can download the file directly from the code (wget need to be installed)
!wget https://github.com/ludovicmoncla/insa-5gi-dsc-tutorials/blob/main/data/data-weather-lyon.csv

In [ ]:
## Load the data into a dataframe
df_weather = pd.read_csv('data-weather-lyon.csv')

In [ ]:
## Display the first rows
df_weather.head()

### 4.2. First overview of the weather data

The dataset contains the following columns:

- WD10M           MERRA-2 Wind Direction at 10 Meters (Degrees) 
- T2M             MERRA-2 Temperature at 2 Meters (C) 
- RH2M            MERRA-2 Relative Humidity at 2 Meters (%) 
- QV2M            MERRA-2 Specific Humidity at 2 Meters (g/kg) 
- T2MDEW          MERRA-2 Dew/Frost Point at 2 Meters (C) 
- U10M            MERRA-2 Eastward Wind at 10 Meters (m/s) 
- PS              MERRA-2 Surface Pressure (kPa) 
- T2MWET          MERRA-2 Wet Bulb Temperature at 2 Meters (C) 
- WS10M           MERRA-2 Wind Speed at 10 Meters (m/s) 
- V10M            MERRA-2 Northward Wind at 10 Meters (m/s) 
- PRECTOTCORR     MERRA-2 Precipitation Corrected (mm/hour) 


Use the method [drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) to delete the columns you don't want to keep.

In [ ]:
## We remove useless columns
## We only want to keep the temperature, wind speed and rainfall columns.

df_weather = *****

df_weather.head()

Use the method [rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) to rename the remaining columns in order to be able to make a join with the vélo'v availability data.

In [ ]:
## Renaming columns

df_weather = *****
df_weather.head()


* Plot the data to see their distribution.

In [ ]:

fig,(ax1,ax2,ax3)= plt.subplots(nrows=3)
fig.set_size_inches(12,20)

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='temperature', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'temperature', ax=ax1)
ax1.set(xlabel='Month', ylabel='Average Temperature', title='Average Temperature by Month') 

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='precipitation', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'precipitation', ax=ax2)
ax2.set(xlabel='Month', ylabel='Average Precipitation', title='Average Precipitation by Month') 

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='vent', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'vent', ax=ax3)
ax3.set(xlabel='Month', ylabel='Average Wind Speed', title='Average Wind Speed by Month') 

Use the [merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method to combine the velo'v availability data with the weather data.

In [ ]:
## On ajoute les données météo dans le dataframe qui contient les données de disponibilité
df_bikes = *****
df_bikes.head()

## 4. Training of a prediction model


### 4.1 Preparation of data for training

In [ ]:
## We make a copy of our DataFrame, to be able to return to the initial data if necessary
df_data = df_bikes.copy()

In [ ]:
# We split the dataset into inputs and outputs for learning
# the input correspond to the data provided to the model in order to learn. 
inputs = *****
# outputs or labels correspond to the values that we want to predict and that the model must learn
labels = *****

Most machine learning methods can only use numerical variables. It is therefore necessary to transform categorical variables such as `id_velov` or `day_of_week` so that they can be used for learning. 

For that, we use the function [LabelEncoder()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) of the scikit-learn library. It allows to transform string values into numerical values by association. For example : Monday -> 0, Tuesday -> 1, etc...

In [ ]:
# Trabnsform categorical variables into numerical variables
# Declare label encoders
l1_encoder = LabelEncoder()
l2_encoder = LabelEncoder()

# We train the encoder to see all possible values
l1_encoder.fit(df_stations.id_velov)

# we transform the id_velov column
inputs['id_velov'] = l1_encoder.transform(inputs['id_velov'])

# We do the same for the day_of_week column
l2_encoder.fit(inputs['day_of_week'])
inputs['day_of_week'] = l2_encoder.transform(inputs['day_of_week'])

# we check the results
inputs.head()

Once the data is ready and the variables are in the right format. Now we have to separate the dataset to use a part of the data for training and another part for evaluation. 

For this we use the method [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) which allows to separate the dataset in 2 random sets so that the training and evaluation datasets have the same characteristics.

In [ ]:
# We separate the dataset in 2: a training set and a test set
x_train, x_test, y_train, y_test = *****


### 4.2 Supervised learning

All the supervised learning algorithms implemented in the Scikit-Learn library are available here: https://scikit-learn.org/stable/supervised_learning.html

In our case, we want to predict continuous values as opposed to discrete values. We will therefore use regression models and not classification models.

I propose you to test and compare 2 methods : [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) et [RandomForestRegressor](https://scikit-learn.org/0.15/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

#### 4.2.1 Linear Regression

In [ ]:
# We declare a model of type RadomForestRegressor and we train it on the training set
clf_lr = *****

*****

In [ ]:
# We use the model to make the prediction on the test set
predictions_lr = *****

#### 4.2.2 Random Forest Regressor

In [ ]:
# We declare a model of type RadomForestRegressor and we train it on the training set
clf_rf = *****
*****

# !! This cell can take several minutes to execute (between 5 and 10 min) !! #

In [ ]:
# We use the model to make the prediction on the test set
predictions_rf = *****

### 4.3 Evaluation and comparison

In order to evaluate our prediction models, we use the [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) (Mean Absolute Error) measure: 
$$\frac{\sum_{i=1}^n \left\lvert \hat{y}_i - y_i \right\rvert}{n}$$ 
which allows us to measure the absolute mean difference between the predictions and the values to be predicted. The lower this value is, the better our model is.


In [ ]:
# The average error is calculated by comparing the predictions with the values that should be predicted

mae_lr = *****

print('linear regression mae : ', mae_lr)

In [ ]:
# The average error is calculated by comparing the predictions with the values that should be predicted

mae_rf = *****

print('random forest regression mae : ', mae_rf)

We observe that the average error is much lower for the random forest model! On average the difference between the prediction and the reality is 1. This seems rather satisfactory for our task. In the case of using this model in an application, the user would have to be warned that the prediction is on average correct to within 1 bike!

We now want to display an evaluation in graphical form. Since the test set contains more than a million rows, we first isolate a sample of 100 data.

In [ ]:
# We group the values to be predicted and the predictions in the same table
t = np.stack((y_test, predictions_rf, predictions_lr), axis=0)

In [ ]:
# We check the size of the table
t.shape

In [ ]:
# We display an overview of the values
t

In [ ]:
# We select a sample of 100 predictions to display the comparison

idx = np.random.randint(t.shape[1], size=100)
sample = t[:,idx]

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Comparison of the  predictions")
plt.xlabel("Sample")
plt.ylabel("# bikes available")
plt.plot(range(100), sample[0], color ="green", label="Truth")
plt.plot(range(100), sample[1], color ="blue", label="Random Forest")
plt.plot(range(100), sample[2], color ="red", label="Linear Regression")
plt.legend()
plt.show()

Now we want to display the same type of graph but instead of displaying the prediction we want to display the error.

In [ ]:
err_rf = abs(sample[0]-sample[1])
err_lr = abs(sample[0]-sample[2])

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Comparison of the  predictions")
plt.xlabel("Sample")
plt.ylabel("Error")
plt.plot(range(100), err_rf, color ="blue", label="Random Forest")
plt.plot(range(100), err_lr, color ="red", label="Linear Regression")
plt.legend()
plt.show()

### 4.4 Using the model

Does the weather affect the prediction?

To find out, we can make a prediction of availability at a station for a certain date by varying the weather and observe if there are differences between the predictions.


The model takes as input 10 parameters:
- id_velov
- year
- month
- day
- hour
- minute
- day_of_week
- temperature
- wind
- precipitation

We propose to make a prediction for next Saturday at 8am at the station 3094 (Gare Part-Dieu). 
We make the prediction for 2 different cases: good and bad weather
1. Good weather : temperature = 20, wind = 1, precipitation = 0
2. Bad weather : temperature = 8, wind = 3, precipitation = 0.10

In [ ]:

# encode the name of the station
st = *****

# encode the name of the day
dn = *****

# list of input data
d_nice = *****
d_bad = *****

# make the prediction
p_nice = *****
p_bad = *****


In [ ]:
print(p_nice)
print(p_bad)

We can see that the weather does have an impact on the prediction. There will be more bikes available in bad weather than in good weather!

## Exercice

Using the code from the previous sessions, propose a map of the availability of vélo'v for all the stations for tomorrow at 8 am.

In [ ]:
# To be completed


    